## ETL Report: 
## Firm Charaterstics Among the Three Largest Municipalities in Florida

### Introduction

We asked ourselves how do the three largest municipalities (Orlando, Miami, Tampa) in Florida compare in terms of firm charaterstics such as employee/firm amounts, diversity, and other KPI's on the region's economy.


This ETL report will go over the process of extracting, transforming, and loading data from two different data sources.
* The United States Census Bureau 
* Federal Reserve Bank of St. Louis 

All steps will be outlined below for the ETL process using Python and the Pandas library.

### Extraction: Part 1 

Our first steps first extracting this data were obtaining the proper API keys to access the sets. 
*(Please note, if you do not plan on querying more 500 times a day, you can skip obtaining the key)
However, to obtain an API key for the Census Bureau, please click on the following [link](https://www.census.gov/data/developers/guidance/api-user-guide.Help_&_Contact_Us.html
)

Once a key is obtained, you will need to query the columns and sets you are interested in, see below for the code used to query the needed data.



In [2]:
import requests
import pandas as pd

First, we import the Pandas Library as pd to work with dataframes and the requests Library to perform the API call


Then
* We use the variable endpoint to define the query
* Then use a GET request to obtain the and store the API
* In the variable df, we covert the response from JSON to a Pandas dataframe
* And finally, to a CSV file to be saved

In [3]:
endpoint = 'https://api.census.gov/data/2019/abscs?get=NAME,GEO_ID,RCPSZFI,NAICS2017_LABEL,ETH_GROUP,SEX,EMPSZFI,EMP,FIRMPDEMP&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:33100&NAICS2017=00'
response = requests.get(endpoint)
df = pd.DataFrame.from_records(response.json()[1:], columns=response.json()[0])
df.to_csv('../assessment_8/data/census_data_miami.csv', index=False)

In [ ]:
endpoint = 'https://api.census.gov/data/2019/abscs?get=NAME,GEO_ID,RCPSZFI,NAICS2017_LABEL,ETH_GROUP,SEX,EMPSZFI,EMP,FIRMPDEMP&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:36740&NAICS2017=00'
response = requests.get(endpoint)
df = pd.DataFrame.from_records(response.json()[1:], columns=response.json()[0])
df.to_csv('../assessment_8/data/census_data_orlando.csv', index=False)

In [ ]:
endpoint = 'https://api.census.gov/data/2019/abscs?get=NAME,GEO_ID,RCPSZFI,NAICS2017_LABEL,ETH_GROUP,SEX,EMPSZFI,EMP,FIRMPDEMP&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:33100&NAICS2017=00'
response = requests.get(endpoint)
df = pd.DataFrame.from_records(response.json()[1:], columns=response.json()[0])
df.to_csv('../assessment_8/data/census_data_tampa.csv', index=False)

*Please be aware, when querying from this spefic API, it will help to use the documentation provided to determine which columns one would like to query, feel free to view them [here](https://www.census.gov/programs-surveys/abs/technical-documentation/api.2019.html#list-tab-702748516). Also, the CSV path may be different depending on your machine, please adjust accordingly.

### Extraction: Part 2

Now, the above was for accessing the data from The United States Census Bureau, below are the steps for extracting the Federal Reserve Bank of St. Louis data sets

Please access the revelent data from these three links:
* [Miami](https://fred.stlouisfed.org/series/RGMP33100)
* [Orlando](https://fred.stlouisfed.org/series/NGMP36740)
* [Tampa](https://fred.stlouisfed.org/series/NGMP45300)

On each link, you will see a dashboard visual for the municipalities, to the right will be a blue download button, please click on it and choose to download as a CSV.

Once all threes CSV files are downloaded, be sure to rename them to the relevant city, and place them within the same working folder as you have the rest of your data and Juypter Noteboook.

### Transformation : Part 1

Now that we have the uncleaned data in its raw form from the API, we can transform it to only contain data relevant to our question

First, create a new varaible to read in the raw CSV data for each municipality, this will allow us to manipulate the data frames as needed

See below:

In [ ]:
census_miami = pd.read_csv('../assessment_8/data/census_data_miami.csv')
census_tampa = pd.read_csv('../assessment_8/data/census_data_tampa.csv')
census_orlando = pd.read_csv('../assessment_8/data/census_data_orlando.csv')

From here, we will need to complete the cleaning steps for each data set, the relvant code is posted below to change columns, values types, and remove any information that is not needed






- Change current column names to more readable names, as well as drop columns outside your needs(complete for each dataframe)

In [ ]:
drop_rename_columns = census_miami.rename(columns={
'NAME':'Municipality', 'RCPSZFI': 'Firm_Sale_Amount', 'ETH_GROUP' : 'Ethnicity', 'SEX': 'Gender',
'EMPSZFI': 'Firm_Employee_Size', 'EMP': 'Employee_Amount', 'FIRMPDEMP': 'Amount_of_Firms'})

check_columns = drop_rename_columns.drop(['NAICS2017_LABEL', 'NAICS2017', 'metropolitan statistical area/micropolitan statistical area', 'Firm_Employee_Size'], axis=1)
check_columns

Next, we will replace values, due to the way the data is preseneted, you will need to reference the documentation for value codes depending on the columns. Values codes can be found using this [link](https://www.census.gov/programs-surveys/abs/technical-documentation/api.2019.html#list-tab-702748516) (same as mentioned earlier)

- Replace value codes with the relevant data

In [ ]:
check_columns.loc[:,'Firm_Sale_Amount'] = check_columns['Firm_Sale_Amount'].astype(str).replace({
'511': 'Less than $5,000', '518': '$5,000,$9,000', '531': '$500,000-$999,999', '519' : ' $10,000-$24,999', 
'521': '$25,000-$49,999', '522' : '$50,000-$99,999', '523': '$100,000-$249,999',
'525': '$250,000 to $499,999', '532': ' $1,000,000 or more' })

* Be sure to keep track of you value types, replacing the incorrect value type will result in an error, reference your dataframe using 
```python
your_data_frame.info()
    

In [ ]:
replaced_values = (check_columns
    .replace({'Gender':{1: 'Total'}})
    .replace({'Gender':{2: 'Female'}})
    .replace({'Gender':{3: 'Male'}})
    .replace({'Gender':{4: 'Equally male/female'}})
    .replace({'Gender':{96: 'Classified'}})
    .replace({'Gender':{98: 'Unclassificable'}})
    .replace({'Ethnicity':{ 1 : 'Total'}})
    .replace({'Ethnicity':{20: 'Hispanic'}})
    .replace({'Ethnicity':{28: 'Equally Hispanic/non-Hispanic'}})
    .replace({'Ethnicity':{29: 'Non-Hispanic'}})
    .replace({'Ethnicity':{96: 'Classifiable'}})
    .replace({'Ethnicity':{98: 'Unclassifiable'}})
)

Now, we will also need to remove rows that do not contain any data, this set uses 'Total' or numeric objects to represent null values
<!-- ![image info](./transform_table.png) -->


Please see below the code needed to replace numeric objects:

- To remove numeric objects

In [ ]:
final_miami = replaced_values.drop(replaced_values[replaced_values['Firm_Sale_Amount'] == '1'].index)

Next, we will add a state column to for reference, as well as replace total with null values, see the code below:

* Adding state of Florida

In [ ]:
final_miami['State'] = pd.Series(['Florida']*139)

At this point, your dataframe should look similar to this snippet below:


![image info](../assessment_8/Images/Screenshot%20(79).png)



Now, we can conduct the final step of concatination of all tables, as well as taking care of null values

* Here, we can replace 'Total' with null values

* *remember, there are three seperate data frames that will need all the above code applied to each one, in this final step we are appending all the dataframes into one large dataframe 


The code below is an exception that can be applied to all three dataframes

In [ ]:
municipality_dataframe = pd.concat([census_miami, census_orlando, census_tampa])
municipality_df = municipality_dataframe.replace({'Total': 'null'})

Your Final table should resemble the one below:

![image info](../assessment_8/Images/Screenshot%20(80).png)


Finally, save the dataframe as a CSV to your data folder, see code below:

In [ ]:
municipality_df.to_csv('../assessment_8/data/municipality_df.csv', index=False)

### Transformation: Part 2


Now, for the FRED economic data sets, there are a couple transformation steps to get the table to normal form

* Read in files
* Merge tables
* rename columns


Please see the code below to read in the files

In [ ]:
orlando_gdp = pd.read_csv('../assessment_8/data/orlando_gpd.csv')
miami_gdp = pd.read_csv('../assessment_8/data/miami_gdp.csv')
tampa_gpd = pd.read_csv('../assessment_8/data/tampa_gdp.csv')
miami_gdp

* Merge all three sets 

In [ ]:
merged = pd.merge(orlando_gdp,miami_gdp, on='DATE', how='left')
df = pd.merge(merged,tampa_gpd, on= 'DATE', how='left')
df

* Rename the columns for easier readability 

In [ ]:
gdp_final= df.rename(columns={'NGMP36740': 'Tampa_GDP', 'RGMP33100': 'Miami_GDP', 'NGMP45300': 'Orlando_GDP'})
gdp_final

Your final dataframe should look like this:



![image info](../assessment_8/Images/Screenshot%20(81).png)


Finally, save the final GDP dataframe to your data folder

In [ ]:
gdp_final.to_csv('../assessment_8/data/gdp_df.csv', index=False)

After all steps are completed, you should be left with two data frames, one from our Census API that has Orlando, Miami, and Tampa municipality business characteristics as well as the GDP over the last decades for each of the municipalities

These data frames will be helpful in creating visuals to compare KPI's from the cities, thus allowing us to answer how well one is doing to the other, and allow us to draw conclusions on why one would be performing better than the other

### Load

Now, the process to load is fairly simple since all ETL steps are completed within Python, please see below:


* Make sure to load the libraries needed for your visualizations, please see the ones used in our process below:

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

*If you do not have any of these libraries installed, please see this [documentation](https://dylancastillo.co/how-to-plot-with-python-popular-graphs-using-pandas-matplotlib-seaborn-and-plotly-express/#start-jupyter-notebook-and-import-libraries) for assistance


* Read the CSV files of the two dataframes into your Juypter Notebook

In [ ]:
municipality_df = pd.read_csv('../assessment_8/data/municipality_df.csv')
gdp_df = pd.read_csv('../assessment_8/data/gdp_df.csv')

### Conclusion

This concludes the ETL process for our data sets, there are many steps involved getting the data normalized to the point of use, however, we feel this ETL Report can be very helpful for obtaining not only the prefered data for our process, but can be a great reference for any data request whether from an API or file. 


If there are additonal questions, feel free to explore the documentation below to assist

* [API_Calling](https://www.nylas.com/blog/use-python-requests-module-rest-apis/)

* [ETL_Process](https://www.nylas.com/blog/use-python-requests-module-rest-apis/)

* [Pandas_CheatSheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)